# CAMO-Net ETL – SILVER → GOLD
### Notebook de inspeção e execução passo a passo

Este notebook reproduz o script `03_silver_to_gold.py` e permite verificar:

- Criação das 6 dimensões:
  - dim_tempo
  - dim_unidade_saude
  - dim_atendimento
  - dim_paciente
  - dim_medicamento
  - dim_diagnostico

- Criação das 3 tabelas fato:
  - fato_prescricao
  - fato_diagnostico
  - fato_atendimento_resumo

- Validação de integridade referencial entre fatos e dimensões

Ideal para depuração, auditoria e entendimento do modelo dimensional.


In [19]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime

# Ajuste conforme a posição deste notebook
project_root = Path.cwd().parent

silver_path = Path('../../data/silver/')
gold_path   = Path('../../data/gold/')


In [20]:
print("Silver:", silver_path, "->", silver_path.exists())

if not silver_path.exists():
    raise FileNotFoundError("A pasta 'silver' não existe. Ajuste o path manualmente.")
    
gold_path.mkdir(parents=True, exist_ok=True)
print("Gold:", gold_path, "-> OK (criada se não existia)")


Silver: ../../data/silver -> True
Gold: ../../data/gold -> OK (criada se não existia)


In [21]:
def calcular_faixa_etaria(idade):
    if pd.isna(idade):
        return 'Não informado'
    
    idade = int(idade)
    if idade < 1:
        return '0-1 ano'
    elif idade < 12:
        return '1-11 anos'
    elif idade < 18:
        return '12-17 anos'
    elif idade < 60:
        return '18-59 anos'
    else:
        return '60+ anos'


In [22]:
def classificar_who_aware(composto_quimico):
    if pd.isna(composto_quimico):
        return 'Not Applicable'
    
    c = str(composto_quimico).upper()

    access = ['AMOXICILINA','AMPICILINA','PENICILINA','DOXICICLINA','CEFALEXINA',
              'SULFAMETOXAZOL','TRIMETOPRIMA','METRONIDAZOL','NITROFURANTOINA','GENTAMICINA']

    watch = ['CIPROFLOXACINO','LEVOFLOXACINO','AZITROMICINA','CLARITROMICINA',
             'CEFTRIAXONA','CEFOTAXIMA','CEFUROXIMA','AMOXICILINA + CLAVULANATO']

    reserve = ['MEROPENEM','IMIPENEM','VANCOMICINA','LINEZOLIDA','COLISTINA',
               'TIGECICLINA','DAPTOMICINA']

    for a in access:
        if a in c:
            return 'Access'
    for a in watch:
        if a in c:
            return 'Watch'
    for a in reserve:
        if a in c:
            return 'Reserve'

    return 'Not Classified'


In [23]:
def classificar_espectro_acao(composto_quimico):
    if pd.isna(composto_quimico):
        return 'Não aplicável'
    
    c = str(composto_quimico).upper()
    
    amplo = ['AMOXICILINA + CLAVULANATO','CIPROFLOXACINO','LEVOFLOXACINO',
             'CEFTRIAXONA','AZITROMICINA','MEROPENEM','IMIPENEM']
    
    estreito = ['PENICILINA','AMOXICILINA','CEFALEXINA','ERITROMICINA',
                'VANCOMICINA','METRONIDAZOL']
    
    for a in amplo:
        if a in c:
            return 'Amplo'
    for a in estreito:
        if a in c:
            return 'Estreito'

    return 'Específico'


In [24]:
def transformar_diagnostico_antibiotico(df):
    # Alterando os valores das colunas para True/False
    df['e_diagnostico_infeccioso'] = df['e_diagnostico_infeccioso'].astype(bool)
    df['e_antibiotico'] = df['e_antibiotico'].astype(bool)
    
    return df

In [25]:
def criar_dim_tempo(silver_path, gold_path):
    print("\n[1/6] dim_tempo")

    atend = pd.read_parquet(silver_path / 'TAB_ATENDIMENTO_ANALISE.parquet')
    atend['data_atendimento'] = pd.to_datetime(atend['data_atendimento'], errors='coerce')

    datas = atend['data_atendimento'].dropna().unique()
    dim = pd.DataFrame({'data_completa': pd.to_datetime(datas)}).sort_values('data_completa')

    dim['sk_tempo'] = range(1, len(dim)+1)
    dim['ano']       = dim['data_completa'].dt.year
    dim['mes']       = dim['data_completa'].dt.month
    dim['trimestre'] = dim['data_completa'].dt.quarter
    dim['semestre']  = dim['data_completa'].dt.month.map(lambda x: 1 if x<=6 else 2)
    dim['dia_semana']= dim['data_completa'].dt.dayofweek
    dim['nome_mes']  = dim['data_completa'].dt.month_name()
    dim['ano_mes']   = dim['data_completa'].dt.to_period('M').astype(str)

    output = gold_path / 'dim_tempo.parquet'
    dim.to_parquet(output, engine='pyarrow', compression='snappy', index=False)
    print(f"  ✓ Registros: {len(dim)}")
    return dim


In [26]:
def criar_dim_unidade_saude(silver_path, gold_path):
    print("\n[2/6] dim_unidade_saude")

    df = pd.read_parquet(silver_path / 'TAB_UNIDADE_SAUDE.parquet')
    dim = df[['cod_unidade_saude', 'nome_unidade', 'tipo','e_analizada']].drop_duplicates()

    dim['sk_unidade_saude'] = range(1, len(dim)+1)
    dim = dim[['sk_unidade_saude','cod_unidade_saude', 'nome_unidade', 'tipo','e_analizada']]

    dim.to_parquet(gold_path/'dim_unidade_saude.parquet', engine='pyarrow', compression='snappy', index=False)
    print(f"  ✓ Registros: {len(dim)}")
    return dim


In [27]:
def criar_dim_atendimento(silver_path, gold_path):
    print("\n[3/6] dim_atendimento")

    df = pd.read_parquet(silver_path/'TAB_ATENDIMENTO_ANALISE.parquet')
    dim = df[['cod_atendimento','especialidade','periodo_extracao']].drop_duplicates()

    dim['sk_atendimento'] = range(1, len(dim)+1)
    dim = dim[['sk_atendimento','cod_atendimento','especialidade','periodo_extracao']]

    dim.to_parquet(gold_path/'dim_atendimento.parquet', engine='pyarrow', compression='snappy', index=False)
    print(f"  ✓ Registros: {len(dim)}")
    return dim


In [28]:
def criar_dim_paciente(silver_path, gold_path):
    print("\n[4/6] dim_paciente")

    df = pd.read_parquet(silver_path/'TAB_ATENDIMENTO_ANALISE.parquet')

    dim = df.groupby('cod_paciente').agg({
        'sexo': lambda x: x.mode()[0] if len(x.mode()) else None,
        'idade': 'mean'
    }).reset_index()

    dim['faixa_etaria'] = dim['idade'].apply(calcular_faixa_etaria)
    dim['idade_anos']   = dim['idade'].round().astype('Int64')

    dim['sk_paciente'] = range(1, len(dim)+1)
    dim = dim[['sk_paciente','cod_paciente','sexo','faixa_etaria','idade_anos']]

    dim.to_parquet(gold_path/'dim_paciente.parquet', engine='pyarrow', compression='snappy', index=False)
    print(f"  ✓ Registros: {len(dim)}")
    return dim


In [36]:
def criar_dim_medicamento(silver_path, gold_path):
    print("\n[5/6] dim_medicamento")

    med = pd.read_parquet(silver_path/'TAB_MEDICAMENTO.parquet')
    dim = med[['cod_medicamento','composto_quimico','tipo_uso','unidade_apresentacao',
               'concentracao','e_antibiotico']].drop_duplicates()

    dim['classe_who_aware'] = dim['composto_quimico'].apply(classificar_who_aware)
    dim['espectro_acao']    = dim.apply(lambda r: classificar_espectro_acao(r['composto_quimico']), axis=1)
    dim['via_administracao']= 'Oral'

    dim['sk_medicamento'] = range(1, len(dim)+1)

    dim.to_parquet(gold_path/'dim_medicamento.parquet', engine='pyarrow', compression='snappy', index=False)
    print(f"  ✓ Registros: {len(dim)}")
    return dim


In [37]:
def criar_dim_diagnostico(silver_path, gold_path):
    print("\n[6/6] dim_diagnostico")

    cid  = pd.read_parquet(silver_path/'TAB_CID_DIAGNOSTICO.parquet')
    ciap = pd.read_parquet(silver_path/'TAB_CIAP_DIAGNOSTICO.parquet')

    dim_cid = cid[['cod_cid','diag_original','diag_agrupado','diag_analise','e_infeccao']].copy()
    dim_cid['tipo_diagnostico']='CID'
    dim_cid = dim_cid.rename(columns={'cod_cid':'codigo_diagnostico'})

    dim_ciap = ciap[['cod_ciap','diag_original','diag_agrupado','diag_analise','e_infeccao']].copy()
    dim_ciap['tipo_diagnostico']='CIAP'
    dim_ciap = dim_ciap.rename(columns={'cod_ciap':'codigo_diagnostico'})

    dim = pd.concat([dim_cid,dim_ciap], ignore_index=True).drop_duplicates('codigo_diagnostico')

    dim['sk_diagnostico'] = range(1, len(dim)+1)

    dim = dim[['sk_diagnostico','codigo_diagnostico','diag_original',
               'diag_agrupado','diag_analise','e_infeccao','tipo_diagnostico']]

    dim.to_parquet(gold_path/'dim_diagnostico.parquet', engine='pyarrow', compression='snappy', index=False)
    print(f"  ✓ Registros: {len(dim)}")
    return dim


In [ ]:
def criar_fato_prescricao(silver_path, gold_path, dimensoes):
    """
    Cria tabela fato de prescrições.
    """
    print("\n[FATO 1/3] Criando fato_prescricao...")
    
    # Desempacotar dimensões
    dim_tempo, dim_unidade, dim_atend, dim_pac, dim_med, dim_diag = dimensoes
    
    # Ler tabelas silver
    med_prescrito = pd.read_parquet(silver_path / 'TAB_MED_PRESCRITO.parquet')
    med_analise = pd.read_parquet(silver_path / 'TAB_MEDPRESCRITO_ANALISE.parquet')
    atend = pd.read_parquet(silver_path / 'TAB_ATENDIMENTO.parquet')
    atend_analise = pd.read_parquet(silver_path / 'TAB_ATENDIMENTO_ANALISE.parquet')
    
    # Base: med_analise (prescrições com análise de antibióticos)
    fato = med_analise.copy()
    
    # Enriquecer com dados de TAB_MED_PRESCRITO
    med_prescrito_cols = med_prescrito[['cod_atendimento', 'cod_medicamento', 'quantidade', 'qtd_receita']].copy()
    fato = fato.merge(
        med_prescrito_cols,
        on=['cod_atendimento', 'cod_medicamento'],
        how='left',
        suffixes=('', '_med_prescrito')
    )
    
    # Usar duração já existente em med_analise
    
    # Join com atendimento (informações contextuais)
    atend_info = atend_analise[['cod_atendimento', 'cod_paciente', 'data_atendimento']].drop_duplicates()
    atend_info['data_atendimento'] = pd.to_datetime(atend_info['data_atendimento'], errors='coerce')
    
    unidade_info = atend[['cod_atendimento', 'cod_unidade_saude']].drop_duplicates()
    
    fato = fato.merge(atend_info, on='cod_atendimento', how='left')
    fato = fato.merge(unidade_info, on='cod_atendimento', how='left')
    
    # Join com diagnóstico (priorizando infecciosos)
    atend_flags = atend_analise.groupby('cod_atendimento')['e_diag_infeccioso'].max().rename('e_diag_infeccioso_agg').reset_index()
    
    atend_analise_sorted = atend_analise.sort_values(['cod_atendimento', 'e_diag_infeccioso'], ascending=[True, False])
    diag_principal = atend_analise_sorted.groupby('cod_atendimento').first().reset_index()
    
    fato = fato.merge(atend_flags, on='cod_atendimento', how='left')
    fato = fato.merge(diag_principal[['cod_atendimento', 'cod_cid_ciap']], on='cod_atendimento', how='left')
    
    # Atualizar flag
    fato['e_diagnostico_infeccioso'] = fato['e_diag_infeccioso_agg']
    fato = fato.drop(columns=['e_diag_infeccioso_agg'])
    
    #conversao para booleano
    # fato = transformar_diagnostico_antibiotico(fato)

    # Substituir business keys → surrogate keys
    
    # Tempo
    fato = fato.merge(
        dim_tempo[['sk_tempo', 'data_completa']],
        left_on='data_atendimento',
        right_on='data_completa',
        how='left'
    ).drop(columns=['data_completa'])
    
    # Unidade
    fato = fato.merge(dim_unidade[['sk_unidade_saude', 'cod_unidade_saude']], on='cod_unidade_saude', how='left')
    
    # Atendimento
    fato = fato.merge(dim_atend[['sk_atendimento', 'cod_atendimento']], on='cod_atendimento', how='left')
    
    # Paciente
    fato = fato.merge(dim_pac[['sk_paciente', 'cod_paciente']], on='cod_paciente', how='left')
    
    # Medicamento
    fato = fato.merge(
        dim_med[['sk_medicamento', 'cod_medicamento', 'tipo_uso', 'espectro_acao', 'classe_who_aware']],
        on='cod_medicamento',
        how='left'
    )
    
    # Diagnóstico
    fato = fato.merge(
        dim_diag[['sk_diagnostico', 'codigo_diagnostico']],
        left_on='cod_cid_ciap',
        right_on='codigo_diagnostico',
        how='left'
    ).drop(columns=['codigo_diagnostico'])
    
    # Flags de prescrição apropriada
    fato['e_prescricao_apropriada'] = (fato['e_antibiotico'] == True) & (fato['e_diagnostico_infeccioso'] == True)
    fato['e_prescricao_inadequada'] = (fato['e_antibiotico'] == True) & (fato['e_diagnostico_infeccioso'] == False)
    
    # Surrogate key final
    fato['sk_prescricao'] = range(1, len(fato) + 1)
    
    # Selecionar colunas finais
    fato_final = fato[[
        'sk_prescricao',
        'sk_atendimento',
        'sk_paciente',
        'sk_medicamento',
        'sk_tempo',
        'sk_unidade_saude',
        'sk_diagnostico',
        'quantidade',
        'qtd_receita',
        'duracao',
        'concentracao',
        'e_antibiotico',
        'e_diagnostico_infeccioso',
        'e_prescricao_apropriada',
        'e_prescricao_inadequada',
        'tipo_uso',
        'espectro_acao',
        'classe_who_aware'
    ]].copy()
    
    # Salvar
    fato_final.to_parquet(gold_path / 'fato_prescricao.parquet', engine='pyarrow', compression='snappy', index=False)
    
    print(f"  ✓ fato_prescricao criada: {len(fato_final):,} registros")
    print(f"    - Antibióticos: {fato_final['e_antibiotico'].sum():,}")
    print(f"    - Prescrições apropriadas: {fato_final['e_prescricao_apropriada'].sum():,}")
    print(f"    - Taxa de adequação: {(fato_final['e_prescricao_apropriada'].sum() / len(fato_final) * 100):.2f}%")
    
    return fato_final


In [39]:
def criar_fato_diagnostico(silver_path, gold_path, dimensoes):
    """
    Cria tabela fato de diagnósticos.
    """
    print("\n[FATO 2/3] Criando fato_diagnostico...")
    
    # Desempacotar dimensões
    dim_tempo, dim_unidade, dim_atend, dim_pac, dim_med, dim_diag = dimensoes
    
    # Ler tabelas
    atend_analise = pd.read_parquet(silver_path / 'TAB_ATENDIMENTO_ANALISE.parquet')
    atend = pd.read_parquet(silver_path / 'TAB_ATENDIMENTO.parquet')
    
    # Base: 1 linha = 1 diagnóstico
    fato = atend_analise.copy()
    fato['data_atendimento'] = pd.to_datetime(fato['data_atendimento'], errors='coerce')
    
    # Pegar cod_unidade
    unidade_info = atend[['cod_atendimento', 'cod_unidade_saude']].drop_duplicates()
    fato = fato.merge(unidade_info, on='cod_atendimento', how='left')
    
    # Substituir business keys → surrogate keys
    fato = fato.merge(dim_tempo[['sk_tempo','data_completa']], left_on='data_atendimento', right_on='data_completa', how='left').drop(columns=['data_completa'])
    fato = fato.merge(dim_unidade[['sk_unidade_saude','cod_unidade_saude']], on='cod_unidade_saude', how='left')
    fato = fato.merge(dim_atend[['sk_atendimento','cod_atendimento']], on='cod_atendimento', how='left')
    fato = fato.merge(dim_pac[['sk_paciente','cod_paciente']], on='cod_paciente', how='left')
    fato = fato.merge(dim_diag[['sk_diagnostico','codigo_diagnostico']], left_on='cod_cid_ciap', right_on='codigo_diagnostico', how='left').drop(columns=['codigo_diagnostico'])
    
    # Surrogate key da própria tabela fato
    fato['sk_diagnostico_atendimento'] = range(1, len(fato) + 1)
    
    fato_final = fato[[
        'sk_diagnostico_atendimento',
        'sk_atendimento',
        'sk_paciente',
        'sk_diagnostico',
        'sk_tempo',
        'sk_unidade_saude',
        'diagnosticar_por',
        'e_diag_infeccioso'
    ]].copy()
    
    fato_final.to_parquet(gold_path / 'fato_diagnostico.parquet', engine='pyarrow', compression='snappy', index=False)
    
    print(f"  ✓ fato_diagnostico criada: {len(fato_final):,} registros")
    print(f"    - Diagnósticos infecciosos: {fato_final['e_diag_infeccioso'].sum():,}")
    
    return fato_final


In [40]:
def criar_fato_atendimento_resumo(silver_path, gold_path, dimensoes):
    """
    Cria tabela fato agregada de atendimentos.
    """
    print("\n[FATO 3/3] Criando fato_atendimento_resumo...")
    
    # Desempacotar dimensões
    dim_tempo, dim_unidade, dim_atend, dim_pac, dim_med, dim_diag = dimensoes
    
    # Ler tabelas
    atend = pd.read_parquet(silver_path / 'TAB_ATENDIMENTO.parquet')
    atend_analise = pd.read_parquet(silver_path / 'TAB_ATENDIMENTO_ANALISE.parquet')
    med_prescrito = pd.read_parquet(silver_path / 'TAB_MED_PRESCRITO.parquet')
    med_analise = pd.read_parquet(silver_path / 'TAB_MEDPRESCRITO_ANALISE.parquet')
    
    # Base: atendimentos únicos
    fato = atend_analise[['cod_atendimento', 'cod_paciente', 'data_atendimento', 'especialidade']].drop_duplicates()
    fato['data_atendimento'] = pd.to_datetime(fato['data_atendimento'], errors='coerce')
    
    # Agregar diagnósticos
    diag_agg = atend_analise.groupby('cod_atendimento').agg({
        'cod_cid_ciap': 'count',
        'e_diag_infeccioso': 'sum'
    }).rename(columns={
        'cod_cid_ciap': 'total_diagnosticos',
        'e_diag_infeccioso': 'total_diagnosticos_infecciosos'
    })
    
    primeiro_diag = atend_analise.groupby('cod_atendimento').first()['cod_cid_ciap']
    
    # Agregar medicamentos
    med_agg = med_prescrito.groupby('cod_atendimento').size().rename('total_medicamentos_prescritos')
    
    # Agregar antibióticos
    atb_agg = med_analise.groupby('cod_atendimento').agg({
        'e_antibiotico': 'sum'
    }).rename(columns={'e_antibiotico': 'total_antibioticos_prescritos'})
    
    # Merge agregações
    fato = fato.merge(diag_agg, on='cod_atendimento', how='left')
    fato = fato.merge(primeiro_diag, on='cod_atendimento', how='left', suffixes=('', '_principal'))
    fato = fato.merge(med_agg, on='cod_atendimento', how='left')
    fato = fato.merge(atb_agg, on='cod_atendimento', how='left')
    
    # cod_unidade
    unidade_info = atend[['cod_atendimento','cod_unidade_saude']].drop_duplicates()
    fato = fato.merge(unidade_info, on='cod_atendimento', how='left')
    
    # Preencher NAs
    fato = fato.fillna({
        'total_diagnosticos': 0,
        'total_diagnosticos_infecciosos': 0,
        'total_medicamentos_prescritos': 0,
        'total_antibioticos_prescritos': 0
    })
    
    # Substituir por surrogate keys
    fato = fato.merge(dim_tempo[['sk_tempo','data_completa']], left_on='data_atendimento', right_on='data_completa', how='left').drop(columns=['data_completa'])
    fato = fato.merge(dim_unidade[['sk_unidade_saude','cod_unidade_saude']], on='cod_unidade_saude', how='left')
    fato = fato.merge(dim_atend[['sk_atendimento','cod_atendimento']], on='cod_atendimento', how='left')
    fato = fato.merge(dim_pac[['sk_paciente','cod_paciente']], on='cod_paciente', how='left')
    fato = fato.merge(dim_diag[['sk_diagnostico','codigo_diagnostico']], left_on='cod_cid_ciap', right_on='codigo_diagnostico', how='left').drop(columns=['codigo_diagnostico'])
    
    # Flags
    fato['teve_prescricao_antibiotico'] = fato['total_antibioticos_prescritos'] > 0
    fato['teve_diagnostico_infeccioso'] = fato['total_diagnosticos_infecciosos'] > 0
    
    fato_final = fato[[
        'sk_atendimento',
        'sk_paciente',
        'sk_tempo',
        'sk_unidade_saude',
        'sk_diagnostico',
        'especialidade',
        'total_diagnosticos',
        'total_medicamentos_prescritos',
        'total_antibioticos_prescritos',
        'total_diagnosticos_infecciosos',
        'teve_prescricao_antibiotico',
        'teve_diagnostico_infeccioso'
    ]].copy()
    
    fato_final.to_parquet(gold_path / 'fato_atendimento_resumo.parquet', engine='pyarrow', compression='snappy', index=False)
    
    print(f"  ✓ fato_atendimento_resumo criada: {len(fato_final):,} registros")
    print(f"    - Com prescrição de antibiótico: {fato_final['teve_prescricao_antibiotico'].sum():,}")
    print(f"    - Com diagnóstico infeccioso: {fato_final['teve_diagnostico_infeccioso'].sum():,}")
    
    return fato_final


In [41]:
def validar_integridade_referencial(gold_path):
    """
    Valida integridade referencial entre fatos e dimensões.
    """
    print("\n" + "="*80)
    print("VALIDAÇÃO DE INTEGRIDADE REFERENCIAL")
    print("="*80)
    
    # Carregar dimensões
    dim_tempo = pd.read_parquet(gold_path / 'dim_tempo.parquet')
    dim_unidade = pd.read_parquet(gold_path / 'dim_unidade_saude.parquet')
    dim_atend = pd.read_parquet(gold_path / 'dim_atendimento.parquet')
    dim_pac = pd.read_parquet(gold_path / 'dim_paciente.parquet')
    dim_med = pd.read_parquet(gold_path / 'dim_medicamento.parquet')
    dim_diag = pd.read_parquet(gold_path / 'dim_diagnostico.parquet')
    
    # Carregar fatos
    fato_presc = pd.read_parquet(gold_path / 'fato_prescricao.parquet')
    fato_diag = pd.read_parquet(gold_path / 'fato_diagnostico.parquet')
    fato_atend = pd.read_parquet(gold_path / 'fato_atendimento_resumo.parquet')
    
    erros = []
    
    # -------------------------
    # 1) Validar fato_prescricao
    # -------------------------
    print("\n[1] Validando fato_prescricao...")
    if not fato_presc['sk_tempo'].dropna().isin(dim_tempo['sk_tempo']).all():
        erros.append("fato_prescricao: FKs inválidas em sk_tempo")
    if not fato_presc['sk_paciente'].dropna().isin(dim_pac['sk_paciente']).all():
        erros.append("fato_prescricao: FKs inválidas em sk_paciente")
    if not fato_presc['sk_medicamento'].dropna().isin(dim_med['sk_medicamento']).all():
        erros.append("fato_prescricao: FKs inválidas em sk_medicamento")
    
    if not any("fato_prescricao" in e for e in erros):
        print("  ✓ Integridade referencial OK em fato_prescricao")
    
    # -------------------------
    # 2) Validar fato_diagnostico
    # -------------------------
    print("\n[2] Validando fato_diagnostico...")
    if not fato_diag['sk_tempo'].dropna().isin(dim_tempo['sk_tempo']).all():
        erros.append("fato_diagnostico: FKs inválidas em sk_tempo")
    if not fato_diag['sk_paciente'].dropna().isin(dim_pac['sk_paciente']).all():
        erros.append("fato_diagnostico: FKs inválidas em sk_paciente")
    
    if not any("fato_diagnostico" in e for e in erros):
        print("  ✓ Integridade referencial OK em fato_diagnostico")
    
    # -------------------------
    # 3) Validar fato_atendimento_resumo
    # -------------------------
    print("\n[3] Validando fato_atendimento_resumo...")
    if not fato_atend['sk_atendimento'].dropna().isin(dim_atend['sk_atendimento']).all():
        erros.append("fato_atendimento_resumo: FKs inválidas em sk_atendimento")
    
    if not any("fato_atendimento_resumo" in e for e in erros):
        print("  ✓ Integridade referencial OK em fato_atendimento_resumo")
    
    # -------------------------
    # Resultado final
    # -------------------------
    if erros:
        print("\n⚠️  ERROS ENCONTRADOS:")
        for erro in erros:
            print(f"  - {erro}")
        return False
    else:
        print("\n✓ TODAS AS VALIDAÇÕES PASSARAM!")
        return True


In [42]:
print("🚀 Iniciando pipeline SILVER → GOLD\n")

# ============================================
# 1) CRIAÇÃO DAS DIMENSÕES
# ============================================

print("📌 Criando dimensões...\n")

dim_tempo   = criar_dim_tempo(silver_path, gold_path)
dim_unidade = criar_dim_unidade_saude(silver_path, gold_path)
dim_atend   = criar_dim_atendimento(silver_path, gold_path)
dim_pac     = criar_dim_paciente(silver_path, gold_path)
dim_med     = criar_dim_medicamento(silver_path, gold_path)
dim_diag    = criar_dim_diagnostico(silver_path, gold_path)

dimensoes = (dim_tempo, dim_unidade, dim_atend, dim_pac, dim_med, dim_diag)

print("\n✅ Dimensões criadas com sucesso.")
print(f"  - dim_tempo: {len(dim_tempo):,}")
print(f"  - dim_unidade_saude: {len(dim_unidade):,}")
print(f"  - dim_atendimento: {len(dim_atend):,}")
print(f"  - dim_paciente: {len(dim_pac):,}")
print(f"  - dim_medicamento: {len(dim_med):,}")
print(f"  - dim_diagnostico: {len(dim_diag):,}")


# ============================================
# 2) CRIAÇÃO DAS TABELAS FATO
# ============================================

print("\n📌 Criando tabelas fato...\n")

fato_prescricao         = criar_fato_prescricao(silver_path, gold_path, dimensoes)
fato_diagnostico        = criar_fato_diagnostico(silver_path, gold_path, dimensoes)
fato_atendimento_resumo = criar_fato_atendimento_resumo(silver_path, gold_path, dimensoes)

print("\n✅ Fatos criados com sucesso.")
print(f"  - fato_prescricao: {len(fato_prescricao):,}")
print(f"  - fato_diagnostico: {len(fato_diagnostico):,}")
print(f"  - fato_atendimento_resumo: {len(fato_atendimento_resumo):,}")


# ============================================
# 3) VALIDAÇÃO DE INTEGRIDADE REFERENCIAL
# ============================================

print("\n📌 Validando integridade referencial do modelo dimensional...\n")

validacao_ok = validar_integridade_referencial(gold_path)

if validacao_ok:
    print("\n🎉 VALIDADO! Modelo dimensional consistente.")
else:
    print("\n⚠️ Foram encontrados problemas de integridade referencial.")


# ============================================
# 4) RESUMO FINAL
# ============================================

print("\n📌 RESUMO FINAL")

print("\nDIMENSÕES:")
print(f"  - dim_tempo: {len(dim_tempo):,}")
print(f"  - dim_unidade_saude: {len(dim_unidade):,}")
print(f"  - dim_atendimento: {len(dim_atend):,}")
print(f"  - dim_paciente: {len(dim_pac):,}")
print(f"  - dim_medicamento: {len(dim_med):,}")
print(f"  - dim_diagnostico: {len(dim_diag):,}")

print("\nFATOS:")
print(f"  - fato_prescricao: {len(fato_prescricao):,}")
print(f"  - fato_diagnostico: {len(fato_diagnostico):,}")
print(f"  - fato_atendimento_resumo: {len(fato_atendimento_resumo):,}")

print("\n🚀 Pipeline SILVER → GOLD executado com sucesso!\n")


🚀 Iniciando pipeline SILVER → GOLD

📌 Criando dimensões...


[1/6] dim_tempo
  ✓ Registros: 221

[2/6] dim_unidade_saude
  ✓ Registros: 50

[3/6] dim_atendimento
  ✓ Registros: 224157

[4/6] dim_paciente
  ✓ Registros: 67023

[5/6] dim_medicamento
  ✓ Registros: 33246

[6/6] dim_diagnostico
  ✓ Registros: 1483

✅ Dimensões criadas com sucesso.
  - dim_tempo: 221
  - dim_unidade_saude: 50
  - dim_atendimento: 224,157
  - dim_paciente: 67,023
  - dim_medicamento: 33,246
  - dim_diagnostico: 1,483

📌 Criando tabelas fato...


[FATO 1/3] Criando fato_prescricao...
  ✓ fato_prescricao criada: 306,318 registros
    - Antibióticos: 8,182
    - Prescrições apropriadas: 3,730
    - Taxa de adequação: 1.22%

[FATO 2/3] Criando fato_diagnostico...
  ✓ fato_diagnostico criada: 298,848 registros
    - Diagnósticos infecciosos: 16,572

[FATO 3/3] Criando fato_atendimento_resumo...
  ✓ fato_atendimento_resumo criada: 224,157 registros
    - Com prescrição de antibiótico: 7,115
    - Com diagnóstico i